In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import GPT2Tokenizer, GPT2Model
import torch
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.optimizers import Adam

# Assuming your DataFrame is named 'df'
df = pd.read_csv('/kaggle/input/dataset/combined_DJIA_NEWS.csv')
# Drop unnecessary columns for training
df.drop(['Unnamed: 0', 'Date'], axis=1, inplace=True)

# Create additional features from numerical data
df['Lowest_Low'] = df['Low'].rolling(window=14).min()
df['Highest_High'] = df['High'].rolling(window=14).max()
df['Stochastic_K'] = ((df['Close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])) * 100
df['Stochastic_D'] = df['Stochastic_K'].rolling(window=3).mean()
df['Momentum'] = df['Close'] - df['Close'].shift(10)
df['Rate_of_Change'] = (df['Close'] / df['Close'].shift(10)) * 100
df['William_R'] = ((df['Highest_High'] - df['Close']) / (df['Highest_High'] - df['Lowest_Low'])) * -100
df['A/D_Oscillator'] = (df['High'] - df['Close'].shift()) / (df['High'] - df['Low'])
df['Close_MA_5'] = df['Close'].rolling(window=5).mean()
df['Disparity_5'] = ((df['Close'] - df['Close_MA_5']) / df['Close_MA_5']) * 100
df = df.drop(columns=['Lowest_Low', 'Highest_High', 'Close_MA_5'])

In [3]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Load pre-trained GPT-2 tokenizer and model from Hugging Face Model Hub
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

import numpy as np

class GPT2EmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        embeddings = []
        for row in X.values:
            # Convert each element to string
            row = [str(element) for element in row]

            # Combine headlines into a single string
            combined_headlines = " ".join(row)

            # Tokenize and get embeddings
            inputs = tokenizer(combined_headlines, return_tensors="pt", truncation=True)
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
            embeddings.append(embedding)

        return np.array(embeddings)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [4]:
# Define the features and the target variable
text_features = ['Top' + str(i) for i in range(1, 26)]
numerical_features = ['Stochastic_K', 'Stochastic_D', 'Momentum', 'Rate_of_Change', 'William_R', 'A/D_Oscillator', 'Disparity_5']
target_variable = 'Label'

# Define the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('text', GPT2EmbeddingTransformer(), text_features),
        ('num', StandardScaler(), numerical_features)
    ]
)

# Separate features and target variable in training and testing sets
X_train = train_df[text_features + numerical_features]
y_train = train_df[target_variable]
X_test = test_df[text_features + numerical_features]
y_test = test_df[target_variable]

# Transform the data
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [6]:
print("Shape of X_train_transformed:", X_train_transformed.shape)


Shape of X_train_transformed: (1590, 775)


**LSTM**

In [8]:
# Reshape the data for LSTM
X_train_transformed = X_train_transformed.reshape((X_train_transformed.shape[0], X_train_transformed.shape[1], 1))
X_test_transformed = X_test_transformed.reshape((X_test_transformed.shape[0], X_test_transformed.shape[1], 1))
model = Sequential()
model.add(LSTM(100, input_shape=(X_train_transformed.shape[1], 1), return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train_transformed, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
predictions = (model.predict(X_test_transformed) > 0.5).astype(int)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:")
print(classification_report(y_test, predictions))

Epoch 1/10
45/45 [==============================] - 7s 69ms/step - loss: nan - accuracy: 0.4724 - val_loss: nan - val_accuracy: 0.5346
Epoch 2/10
45/45 [==============================] - 2s 46ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 3/10
45/45 [==============================] - 2s 46ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 4/10
45/45 [==============================] - 2s 47ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 5/10
45/45 [==============================] - 2s 47ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 6/10
45/45 [==============================] - 2s 47ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 7/10
45/45 [==============================] - 2s 47ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 8/10
45/45 [==============================] - 2s 

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

# Assuming your X_train_transformed and X_test_transformed are 3D arrays
# (number of samples, number of timesteps, number of features)
# You can adjust input_shape based on your data

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train_transformed.shape[1], X_train_transformed.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_transformed, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
predictions = (model.predict(X_test_transformed) > 0.5).astype(int)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:")
print(classification_report(y_test, predictions))


Epoch 1/10
45/45 [==============================] - 5s 40ms/step - loss: nan - accuracy: 0.4626 - val_loss: nan - val_accuracy: 0.5346
Epoch 2/10
45/45 [==============================] - 1s 24ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 3/10
45/45 [==============================] - 1s 24ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 4/10
45/45 [==============================] - 1s 24ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 5/10
45/45 [==============================] - 1s 25ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 6/10
45/45 [==============================] - 1s 24ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 7/10
45/45 [==============================] - 1s 24ms/step - loss: nan - accuracy: 0.4647 - val_loss: nan - val_accuracy: 0.5346
Epoch 8/10
45/45 [==============================] - 1s 

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**LogisticRegression**

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train_flattened = imputer.fit_transform(X_train_flattened)
X_test_flattened = imputer.transform(X_test_flattened)


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Assuming X_train_flattened and X_test_flattened are your feature matrices
# and y_train is your target variable

# Impute NaN values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_flattened)
X_test_imputed = imputer.transform(X_test_flattened)

# Define the Logistic Regression model
logreg_model = LogisticRegression(random_state=42)

# Train the model
logreg_model.fit(X_train_imputed, y_train)

# Make predictions on training set
logreg_train_predictions = logreg_model.predict(X_train_imputed)

# Make predictions on test set
logreg_test_predictions = logreg_model.predict(X_test_imputed)

# Evaluate the model on training set
train_accuracy = accuracy_score(y_train, logreg_train_predictions)
print(f"Training Accuracy: {train_accuracy:.2f}")

# Evaluate the model on test set
test_accuracy = accuracy_score(y_test, logreg_test_predictions)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Display classification report for test set
print("Classification Report (Test Set):")
print(classification_report(y_test, logreg_test_predictions))



Training Accuracy: 0.92
Test Accuracy: 0.87
Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       174
           1       0.88      0.88      0.88       224

    accuracy                           0.87       398
   macro avg       0.86      0.87      0.86       398
weighted avg       0.87      0.87      0.87       398



/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**SVM**

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer



# Impute NaN values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_flattened)
X_test_imputed = imputer.transform(X_test_flattened)

# Define the SVM model
svm_model = SVC(random_state=42)

# Train the model
svm_model.fit(X_train_imputed, y_train)

# Make predictions on training set
svm_train_predictions = svm_model.predict(X_train_imputed)

# Make predictions on test set
svm_test_predictions = svm_model.predict(X_test_imputed)

# Evaluate the model on training set
train_accuracy = accuracy_score(y_train, svm_train_predictions)
print(f"Training Accuracy: {train_accuracy:.2f}")

# Evaluate the model on test set
test_accuracy = accuracy_score(y_test, svm_test_predictions)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Display classification report for test set
print("Classification Report (Test Set):")
print(classification_report(y_test, svm_test_predictions))


Training Accuracy: 0.53
Test Accuracy: 0.56
Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       174
           1       0.56      1.00      0.72       224

    accuracy                           0.56       398
   macro avg       0.28      0.50      0.36       398
weighted avg       0.32      0.56      0.41       398



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**svm with window size 25**

In [20]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Assuming X_train_flattened and X_test_flattened are your feature matrices
# and y_train is your target variable

# Impute NaN values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_flattened)
X_test_imputed = imputer.transform(X_test_flattened)

# Set the training window size
window_size = 25

# Initialize the SVM model
svm_model = SVC(random_state=42)

# Train the model with a sliding window
for i in range(window_size, len(X_train_imputed)):
    X_train_window = X_train_imputed[i - window_size:i]
    y_train_window = y_train[i - window_size:i]
    svm_model.fit(X_train_window, y_train_window)

# Make predictions on training set
svm_train_predictions = svm_model.predict(X_train_imputed)

# Make predictions on test set
svm_test_predictions = svm_model.predict(X_test_imputed)

# Evaluate the model on training set
train_accuracy = accuracy_score(y_train, svm_train_predictions)
print(f"Training Accuracy: {train_accuracy:.2f}")

# Evaluate the model on test set
test_accuracy = accuracy_score(y_test, svm_test_predictions)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Display classification report for test set
print("Classification Report (Test Set):")
print(classification_report(y_test, svm_test_predictions))


Training Accuracy: 0.47
Test Accuracy: 0.44
Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.44      1.00      0.61       174
           1       0.00      0.00      0.00       224

    accuracy                           0.44       398
   macro avg       0.22      0.50      0.30       398
weighted avg       0.19      0.44      0.27       398



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**CNN-LSTM**

In [24]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Flatten the data
X_train_flattened = X_train_transformed.reshape(X_train_transformed.shape[0], -1)
X_test_flattened = X_test_transformed.reshape(X_test_transformed.shape[0], -1)

# Impute NaN values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_flattened)
X_test_imputed = imputer.transform(X_test_flattened)

# Reshape back to the original shape
X_train_imputed = X_train_imputed.reshape(X_train_transformed.shape)
X_test_imputed = X_test_imputed.reshape(X_test_transformed.shape)

# CNN-LSTM model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_imputed.shape[1], X_train_imputed.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_imputed, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Make predictions
predictions = (model.predict(X_test_imputed) > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:")
print(classification_report(y_test, predictions))



Epoch 1/10
45/45 [==============================] - 28s 550ms/step - loss: 4.2450 - accuracy: 0.6478 - val_loss: 0.6658 - val_accuracy: 0.6541
Epoch 2/10
45/45 [==============================] - 24s 534ms/step - loss: 0.6636 - accuracy: 0.6653 - val_loss: 0.6655 - val_accuracy: 0.6604
Epoch 3/10
45/45 [==============================] - 25s 549ms/step - loss: 0.6628 - accuracy: 0.6618 - val_loss: 0.6651 - val_accuracy: 0.6415
Epoch 4/10
45/45 [==============================] - 24s 536ms/step - loss: 0.6619 - accuracy: 0.6597 - val_loss: 0.6648 - val_accuracy: 0.6415
Epoch 5/10
45/45 [==============================] - 24s 539ms/step - loss: 0.6611 - accuracy: 0.6583 - val_loss: 0.6642 - val_accuracy: 0.6352
Epoch 6/10
45/45 [==============================] - 24s 543ms/step - loss: 0.6602 - accuracy: 0.6583 - val_loss: 0.6638 - val_accuracy: 0.6352
Epoch 7/10
45/45 [==============================] - 25s 548ms/step - loss: 0.6593 - accuracy: 0.6618 - val_loss: 0.6633 - val_accuracy: 0.6352

**CNN-GRU**

In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Assuming X_train_transformed, X_test_transformed, y_train, and y_test are your data
# X_train_transformed and X_test_transformed should have shape (samples, timesteps, features)

# Flatten the data
X_train_flattened = X_train_transformed.reshape(X_train_transformed.shape[0], -1)
X_test_flattened = X_test_transformed.reshape(X_test_transformed.shape[0], -1)

# Impute NaN values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_flattened)
X_test_imputed = imputer.transform(X_test_flattened)

# Reshape back to the original shape
X_train_imputed = X_train_imputed.reshape(X_train_transformed.shape)
X_test_imputed = X_test_imputed.reshape(X_test_transformed.shape)

# CNN-GRU model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_imputed.shape[1], X_train_imputed.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(GRU(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_imputed, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Make predictions
predictions = (model.predict(X_test_imputed) > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:")
print(classification_report(y_test, predictions))


Epoch 1/10
45/45 [==============================] - 31s 628ms/step - loss: 0.6155 - accuracy: 0.6758 - val_loss: 0.5182 - val_accuracy: 0.7547
Epoch 2/10
45/45 [==============================] - 27s 608ms/step - loss: 0.4704 - accuracy: 0.8148 - val_loss: 0.3466 - val_accuracy: 0.8742
Epoch 3/10
45/45 [==============================] - 27s 606ms/step - loss: 0.3341 - accuracy: 0.8526 - val_loss: 0.3102 - val_accuracy: 0.8805
Epoch 4/10
45/45 [==============================] - 27s 607ms/step - loss: 0.3073 - accuracy: 0.8637 - val_loss: 0.2847 - val_accuracy: 0.8994
Epoch 5/10
45/45 [==============================] - 27s 601ms/step - loss: 0.2793 - accuracy: 0.8728 - val_loss: 0.2442 - val_accuracy: 0.9057
Epoch 6/10
45/45 [==============================] - 27s 607ms/step - loss: 0.2636 - accuracy: 0.8819 - val_loss: 0.2413 - val_accuracy: 0.9057
Epoch 7/10
45/45 [==============================] - 27s 598ms/step - loss: 0.2646 - accuracy: 0.8770 - val_loss: 0.2277 - val_accuracy: 0.9057

**ANN**

In [26]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# Assuming X_train_transformed, X_test_transformed, y_train, and y_test are your data
# X_train_transformed and X_test_transformed should have shape (samples, features)

# Flatten the data
X_train_flattened = X_train_transformed.reshape(X_train_transformed.shape[0], -1)
X_test_flattened = X_test_transformed.reshape(X_test_transformed.shape[0], -1)

# Impute NaN values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_flattened)
X_test_imputed = imputer.transform(X_test_flattened)

# Create a simple ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_imputed.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_imputed, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Make predictions
predictions = (model.predict(X_test_imputed) > 0.5).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:")
print(classification_report(y_test, predictions))


Epoch 1/10
45/45 [==============================] - 2s 9ms/step - loss: 0.7163 - accuracy: 0.6059 - val_loss: 0.6324 - val_accuracy: 0.5660
Epoch 2/10
45/45 [==============================] - 0s 4ms/step - loss: 0.5582 - accuracy: 0.7149 - val_loss: 0.6122 - val_accuracy: 0.6226
Epoch 3/10
45/45 [==============================] - 0s 4ms/step - loss: 0.5257 - accuracy: 0.7407 - val_loss: 0.4689 - val_accuracy: 0.8113
Epoch 4/10
45/45 [==============================] - 0s 4ms/step - loss: 0.4287 - accuracy: 0.8288 - val_loss: 0.4667 - val_accuracy: 0.7421
Epoch 5/10
45/45 [==============================] - 0s 4ms/step - loss: 0.3957 - accuracy: 0.8379 - val_loss: 0.3738 - val_accuracy: 0.8805
Epoch 6/10
45/45 [==============================] - 0s 4ms/step - loss: 0.3562 - accuracy: 0.8616 - val_loss: 0.3440 - val_accuracy: 0.8868
Epoch 7/10
45/45 [==============================] - 0s 4ms/step - loss: 0.3333 - accuracy: 0.8616 - val_loss: 0.3334 - val_accuracy: 0.8553
Epoch 8/10
45/45 [==